# Accessing Through API

### Chosen Models

### OpenAI GPT 4o - Mini 

[4o Mini Documentation](https://platform.openai.com/docs/models#gpt-4o-mini)   
[GPT 4o Model Card](https://openai.com/index/gpt-4o-system-card/)  
[Scoring Evaluation Metrics](https://deepgram.com/learn/arc-llm-benchmark-guide)  
[LangChain OpenAI Chat Models](https://python.langchain.com/docs/integrations/chat/openai/)

**Capabilities**

According to the documentation the GPT-4 Models are high performance models and as of 2023, with roughly ~8B parameters. It the state of the art flag ship model for their market for small projects. 

### MistralAI Nemo Model

[NeMo Model Documentation](https://mistral.ai/news/mistral-nemo/)  
[Other Mistral Models](https://docs.mistral.ai/getting-started/models/models_overview/)  
[LangChain Mistral Chat Model ](https://python.langchain.com/docs/integrations/chat/mistralai/#invocation)



Mistral Nemo is the best small model, state of the art model with 12B. The Mistral documentation even boasts about a new tokenizer used for this model specifically **Tekken** which outperformed it's previous models by ~30%. Compared to Llama 3's tokenizer, NeMo's 

**Install Dependencies**

In [45]:
pip install python-dotenv


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [95]:
pip install -qU langchain-openai langchain-mistralai langchain_core


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [47]:
from dotenv import load_dotenv
import os

# loads the .env file with LLM API Keys
load_dotenv()

True

In [70]:
# sets API access keys
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")
OPEN_AI_API_KEY = os.getenv("OPEN_AI_API_KEY")

#### Creating OpenAI LangChain Model

In [ ]:
from langchain_openai import ChatOpenAI


open_ai_gpt_4o_mini = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=1, 
    api_key=OPEN_AI_API_KEY,
    timeout=30
)

# establish a system prompt with a test message

open_ai_test_messages = [
    (
        "system",
        """
        You're a helpful assistant, you love Portland Oregon, 
        you're an expert of the city and know all about the parks, roads, and attractions and tourism.

        If you don't know something you respond with, 
        "I don't really know, but Portland is a hella weird place, the locals pride themselves on staying creative and bizarre."
        """
    )
]


**Test LangChain OpenAI Implementation**

In [ ]:
# create a test message

test_prompt = ("human", "Where's the best tacos in PDX?")

# add the the message
open_ai_test_messages.append(test_prompt)


# attempt to get a response from the OpenAI model
open_ai_message = open_ai_gpt_4o_mini.invoke(open_ai_test_messages)

**Determine Open AI model's Response**

In [229]:
print(open_ai_message.content)

0


#### Creating Mistral Nemo LangChain Model

In [ ]:
from langchain_mistralai import ChatMistralAI

mistral_nemo = ChatMistralAI(
    model="open-mistral-nemo",
    temperature=0,
    max_tokens=1,
    api_key=MISTRAL_API_KEY,
    timeout=30
)

nemo_test_messages = [
    (
        "system",
        """
        You're a helpful assistant, you love Portland Oregon, 
        you're an expert of the city and know all about the parks, roads, and attractions and tourism.

        If you don't know something you respond with, 
        "I don't really know, but Portland is a hella weird place, the locals pride themselves on staying creative and bizarre."
        """
    )
]

**Test LangChain Mistral Implementation**

In [ ]:
test_prompt = ("human", "Where's a decent sushi place near Burnside?")

# append the positive tweet
nemo_test_messages.append(test_prompt)

# invoke the model to get a response
model_resp = mistral_nemo.invoke(nemo_test_messages)


**Determine Mistral Model's Response**

In [232]:
print(model_resp.content)

0


**Building a LangChain Prompt Template**

The process of building [prompt_templates](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html) with LangChain is fairly easy for a simple prompt like the ones needed for this task. They can grow exponentially with chaining and using LangChains `LangChain Expression Language` or `LCEL`. below we will build a simple template that will be leveraged for any task. For this part it's easy to reuse the same technique we used before for the system prompt in the initial test of the langchain models for both OpenAI and Mistral. This also, conveniently ensures some amount of consistency for both models. simply passing a `{city}`, `{state}`, a `{question}` is all that is needed. This will all come together when we build our [chain](https://python.langchain.com/docs/how_to/sequence/) to invoke when both using the prompts, the invocation will look something like the implementation below:

```python
# tweet chain's invoke method will take arguments as a dict: { "city": <city>, "state": <state>, "question": <question>}
# this will chain the response of the build ChatTemplate to the next element in the chain the llm
# then the response will get chained to the StrOutParser() for evaluation
basic_prompt_chain = prompt_template | llm | StrOutputParser()
```


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt_template = ChatPromptTemplate([
    ("system", """
        You're a helpful assistant, you know all there is to know about {city} in {state}.

        you will only suggest places to eat, parks to visit, and must see attractions around town. 
        """),
    ("human", "{question}: ")
])


#### Test the ChatPromptTemplate with Models

**OpenAI GPT 4o mini test using ChatPromptTemplate**

In [ ]:
from langchain_core.output_parsers import StrOutputParser

# build the open_ai_4o_mini chain
open_ai_prompt_chain = prompt_template | open_ai_gpt_4o_mini | StrOutputParser()

# invoke the chain with a test prompt

# test the prompt chain
open_ai_prompt_chain.invoke({"city": "Louisville", "state": "Kentucky", "question": "Where are some of the best parks?"})


'1'

**Mistral Nemo test using the ChatPromptTemplate**

In [ ]:
from langchain_core.output_parsers import StrOutputParser

# build the open_ai_4o_mini chain
mistral_nemo_prompt_chain = prompt_template | mistral_nemo | StrOutputParser()

# invoke the chain with a test tweet 

# test the prompt chain
mistral_nemo_prompt_chain.invoke({"city": "New Albany", "state": "Indiana", "question": "What are some of the best pizza places?"})


'1'

**Mistral Nemo Native ChatPromptTemplate**

In [246]:
from langchain_core.output_parsers import StrOutputParser

# build the open_ai_4o_mini chain
mistral_nemo_tweet_native_sentiment_chain = native_language_prompt_template | mistral_nemo | StrOutputParser()

# invoke the chain with a test tweet 

# spanish_tweet:  "Estoy triste porque no puedo hablar español con fluidez."
# translation: "I am sad I can't speak spanish fluently"
mistral_nemo_tweet_native_sentiment_chain.invoke({"system_prompt": mistral_spanish_sys_prompt, "tweet": "Estoy triste porque no puedo hablar español con fluidez."})


'1'